![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control
    
    Items.xml: Raw ingredient Items
    Ingredients.xml: Ingredients/Preps that goes into prep/products recipes
    Preps.xml: Preps that go into final product recipes
    Products.xml: Final Recipes/Menu Items
    Conversions.xml: Unit Conversion Information

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy.

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during the ingredients processing process will be ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process

* We exclude the GHG emission from sauces/dressing that has no dominant ingredients

## Import Libraries

In [1]:
#!pip3 install pdpipe

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os

import csv
from itertools import islice

In [3]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

In [4]:
import openpyxl

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

## Load Data

#### Data Path

In [6]:
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "oc", "*.oc"))

In [7]:
filepath_list

['/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03122021_0915.oc']

#### Load Items List

In [8]:
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/items.xml')
    xroot = xtree.getroot()
    for item in xtree.iterfind('Item'):
        ItemId.append(item.attrib['id'])
        Description.append(item.findtext('Description'))
        CaseQty.append(item.findtext('CaseQty'))
        CaseUOM.append(item.findtext('CaseUOM'))
        PakQty.append(item.findtext('PakQty'))
        PakUOM.append(item.findtext('PakUOM'))
        InventoryGroup.append(item.findtext('InventoryGroup'))

        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    ).drop_duplicates()

Items.reset_index(drop=True, inplace=True)

In [9]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.000,ea,1.000,lb,SPICES
1,I-43652,ANCHO CHILLI,1.200,Kg,1.000,Kg,FOOD - GROCERY
2,I-4472,AVOCADO MX,20.000,CT,1.000,CT,PRODUCE
3,I-4973,AVOCADO PULP CHUNKY,12.000,bag,454.000,g,PRODUCE
4,I-4496,BAK CHOY BABY BC,30.000,lb,1.000,lb,PRODUCE


In [10]:
Items.shape

(149, 7)

In [11]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [12]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

#### Load Ingredient List

In [13]:
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Ingredients.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Ingredient'):
        IngredientId.append(x.attrib['ingredient'])
        Conversion.append(x.attrib['conversion'])
        InvFactor.append(x.attrib['invFactor'])
        Qty.append(x.attrib['qty'])
        Recipe.append(x.attrib['recipe'])
        Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()

Ingredients.reset_index(drop=True, inplace=True)

In [14]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3388,1.000,L,1.00000000,0.3058,P-10496
1,I-4660,2.270,Kg,2.20462000,0.6942,P-10496
2,I-4679,1.000,BUNCH,1.00000000,0.0063,P-18318
3,I-4793,10.000,Kg,2.20462000,1.2048,P-18746
4,I-3643,225.000,g,0.00100000,0.1837,P-18907


In [15]:
Ingredients.shape

(629, 6)

In [16]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [17]:
path = os.path.join(os.getcwd(), "data", "processed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

#### Load Preps List

In [18]:
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Preps.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Prep'):
        PrepId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        PakQty.append(x.findtext('PakQty'))
        PakUOM.append(x.findtext('PakUOM'))
        InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()

Preps.reset_index(drop=True, inplace=True)

In [19]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-54666,BAKED|Pasta|Chicken Alfredo,16.000,each,
1,P-56398,BATCH| Guacamole AR,2.500,Kg,PREP
2,P-26134,BATCH|Pibil Pulled Pork,2.722,Kg,PREP
3,P-42364,BB|Mexican Cabbage,1.000,Kg,PREP
4,P-44335,BB|White Bean Burrito Base,10.400,Kg,PREP


In [20]:
Preps.shape

(113, 5)

In [21]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [22]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

#### Load Product List

In [23]:
ProdId = []
Description = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Products.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Prod'):
        ProdId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description}).drop_duplicates()

Products.reset_index(drop=True, inplace=True)

In [24]:
Products

,ProdId,Description
0,R-50859,ALF|Flatbread|Bruschetta
1,R-50494,ALF|Flatbread|Proscuitto
2,R-56505,ALF|Pasta|Chicken Alfredo
3,R-61129,AVOCADO SPINACH PASTA
4,R-44308,BNO|Bowl Pulled Pork
5,R-44315,BNO|Bowl|Ancho Chicken
6,R-44310,BNO|Bowl|Steak
7,R-43587,BNO|Burrito|Ancho Chicken
8,R-44303,BNO|Burrito|Pulled Pork
9,R-43580,BNO|Burrito|Steak


In [25]:
Products.shape

(20, 2)

In [26]:
Products.dtypes

ProdId         object
Description    object
dtype: object

In [27]:
path = os.path.join(os.getcwd(), "data", "processed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

#### Load Conversion List

In [28]:
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Conversions.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Conversion'):
        ConversionId.append(x.attrib['id'])
        Multiplier.append(x.attrib['multiplier'])
        ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
        ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
        ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
        ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [29]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L


In [30]:
Conversions.shape

(128, 6)

In [31]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [32]:
path = os.path.join(os.getcwd(), "data", "processed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

#### Load Cool Food Calculator Emission Factors

In [33]:
cfc_filename = os.path.join(os.getcwd(), "data", "raw", "cool_food_ghg.csv")
cfc = pd.read_csv(cfc_filename)

In [34]:
cfc.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [35]:
cfc.shape

(54, 3)

## Data Cleaning

#### Unit Conversion

In [36]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

for index in Preps.index:
    if Preps.loc[index,'PakUOM'] == 'Kg':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*1000
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'lb':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*453.592
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'oz':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*28.3495
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'L':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*1000
        Preps.loc[index,'StdUom'] = 'ml'
    else: 
        Preps.loc[index,'StdQty'] = Preps.loc[index,'PakQty']
        Preps.loc[index,'StdUom'] = Preps.loc[index,'PakUOM']
        
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54666,BAKED|Pasta|Chicken Alfredo,16.000,each,,16.000,each
1,P-56398,BATCH| Guacamole AR,2.500,Kg,PREP,2500,g
2,P-26134,BATCH|Pibil Pulled Pork,2.722,Kg,PREP,2722,g
3,P-42364,BB|Mexican Cabbage,1.000,Kg,PREP,1000,g
4,P-44335,BB|White Bean Burrito Base,10.400,Kg,PREP,10400,g


In [37]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

## Data Analysis

#### Add New Items to Database

In [38]:
# Load current Items List with assigned Cool Food Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv"))

In [39]:
col_names = list(Items.columns.values)

In [40]:
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in str(Items_Assigned['ItemId']):
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [41]:
New_Items.insert(1, "CategoryID", '')

In [42]:
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


#### Mapping Items to GHG Factors

In [43]:
df1 = pd.DataFrame(Items_Assigned)
df2 = pd.DataFrame(cfc)

In [44]:
df1.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT
1,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
4,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT


In [45]:
df2.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [46]:
mapping = pd.merge(df1, df2.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

In [47]:
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0

In [48]:
mapping.loc[337,'Category ID'].dtype

dtype('float64')

In [49]:
mapping.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,1.0,beef & buffalo meat,41.3463
1,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,1.0,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,1.0,beef & buffalo meat,41.3463
3,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,1.0,beef & buffalo meat,41.3463
4,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT,1.0,beef & buffalo meat,41.3463


In [50]:
mapping.shape

(390, 11)

In [51]:
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Category ID                                               float64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

#### GHG Factors Calculation for Preps

In [52]:
Preps['GHG Emission/g'] = np.nan

In [53]:
for index, row in Preps.iterrows():
    ghgf = 0
    Preps.loc[index, 'GHG Emission/g'] = ghgf

In [54]:
def get_items_prep_ghgf(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghgf = Preps.loc[index, 'GHG Emission/g']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            if ingres.loc[idx,'Uom'] in ['g', 'ml', 'Kg']:
                weight = float(ingres.loc[idx,'Conversion'])*float(ingres.loc[idx,'InvFactor'])
                ghgf += weight*float(ghge)
    Preps.loc[index, 'GHG Emission/g'] = float(ghgf)

In [55]:
for index, row in Preps.iterrows():
    get_items_prep_ghgf(index , row)

In [56]:
Preps#.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission/g
0,P-54666,BAKED|Pasta|Chicken Alfredo,16.000,each,,16.000,each,0.521172
1,P-56398,BATCH| Guacamole AR,2.500,Kg,PREP,2500,g,0.000000
2,P-26134,BATCH|Pibil Pulled Pork,2.722,Kg,PREP,2722,g,32.245294
3,P-42364,BB|Mexican Cabbage,1.000,Kg,PREP,1000,g,34.567209
4,P-44335,BB|White Bean Burrito Base,10.400,Kg,PREP,10400,g,77.661877
5,P-56422,BNO|Fajita Veg,2.000,Kg,,2000,g,26.476185
6,P-50500,CHIFFONADE|Basil,190.000,g,,190.000,g,0.001167
7,P-24750,CHOPPED|Cilantro,0.500,Kg,,500,g,0.000000
8,P-51214,COOKED|Ancho Chicken|Batch,8.000,Kg,PREP,8000,g,11.750209
9,P-41574,COOKED|Black Beans,30.000,Kg,PREP,30000,g,97.273542


### GHG Calculation for Products

In [57]:
Products['GHG Emission (g)'] = np.nan

In [58]:
for index, row in Products.iterrows():
    ghg = 0
    Products.loc[index, 'GHG Emission (g)'] = ghg

In [59]:
ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[1,'ProdId']]
ingres

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
593,I-1120,1.000,CT,1.00000000,1.0000,R-50494
594,I-13254,30.000,g,0.00220462,1.0000,R-50494
595,I-14179,30.000,g,0.00100000,1.0000,R-50494
596,I-17203,5.000,ml,0.00100000,1.0000,R-50494
597,I-2311,35.000,g,0.00100000,1.0000,R-50494
598,I-2697,15.000,g,1.00000000,1.0000,R-50494
599,P-26068,10.000,g,0.00100000,1.0000,R-50494
600,P-50689,50.000,g,1.00000000,1.0000,R-50494


In [60]:
ghg = 0
for idx, row in ingres.iterrows():
    ingre = ingres.loc[idx,'IngredientId']
    if ingre.startswith('I'):
        ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
        if (ingres.loc[idx,'Uom'] == 'g') or (ingres.loc[idx,'Uom'] == 'ml'):
            qty = float(ingres.loc[idx,'Qty'])
            ghg += qty*float(ghge)   
        print(ghge)
        print(ghg)
ghg

78    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
0
146    0.622
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
18.66
35    8.9104
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
285.972
327    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
285.972
9    9.8315
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
630.0745
370    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
630.0745


630.0745

In [61]:
def get_items_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            if (ingres.loc[idx,'Uom'] == 'g') or (ingres.loc[idx,'Uom'] == 'ml'):
                qty = float(ingres.loc[idx,'Qty'])
                ghg += qty*float(ghge)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)

In [62]:
#def get_preps_ghge(index, row):
    

In [63]:
for index, row in Products.iterrows():
    get_items_ghge(index , row)

In [64]:
Products

,ProdId,Description,GHG Emission (g)
0,R-50859,ALF|Flatbread|Bruschetta,273.5320
1,R-50494,ALF|Flatbread|Proscuitto,630.0745
2,R-56505,ALF|Pasta|Chicken Alfredo,0.0000
3,R-61129,AVOCADO SPINACH PASTA,0.0000
4,R-44308,BNO|Bowl Pulled Pork,0.0000
5,R-44315,BNO|Bowl|Ancho Chicken,0.0000
6,R-44310,BNO|Bowl|Steak,356.4160
7,R-43587,BNO|Burrito|Ancho Chicken,0.0000
8,R-44303,BNO|Burrito|Pulled Pork,0.0000
9,R-43580,BNO|Burrito|Steak,356.4160


In [65]:
path = os.path.join(os.getcwd(), "data", "final", "Recipes GHG.csv")
Products.to_csv(path, index = False, header = True)